## Import

In [1]:
import sys
sys.path.append(r'C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\src\facility_location_Bergen\custome_modules')

In [2]:
import warnings
from shapely.errors import ShapelyDeprecationWarning
# Ignore the ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [3]:
import os
import dill
import pyproj
import folium
import requests
import numpy as np
import pandas as pd
import pickle as pkl
import networkx as nx
from scipy import stats
import plotly.graph_objects as go
from sklearn.utils import resample
import cartopy.io.img_tiles as cimgt
from plotly.subplots import make_subplots
from urllib.request import urlopen, Request
from log import print_INFO_message_timestamp, print_INFO_message
from facility_location import AdjacencyMatrix, FacilityLocation, FacilityLocationReport

## Solution analysis

### Loading data

#### Exact solution

In [4]:
times = ["all_day", "all_day_free_flow", "morning", "midday", "afternoon"]
facilities_number = 3

In [5]:
print_INFO_message_timestamp("Loading exact solutions...")

fls_exact = {}

for time in times[:1]:
    print_INFO_message(f"Loading exact solution for {time}")
    path = rf"C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\07_model_output\{facilities_number}_locations\deterministic_exact_solutions\exact_solution_{time}.pkl"
    fls_exact[time] = FacilityLocation.load(path)

[06/19/23 12:41:59] INFO     Loading exact solutions...
                    INFO     Loading exact solution for all_day


In [6]:
with open("prova.txt", "w") as f:
    fls_exact["all_day"].instance.pprint(ostream=f)

#### dfs for solution comparison

In [ ]:
root = r"C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\08_reporting"
paths = [p for p in os.listdir(root) if ("solution_vs_scenario" in p) and ("worst" not in p)]
paths_worst = [p for p in os.listdir(root) if ("solution_vs_scenario" in p) and ("worst" in p)]

In [ ]:
paths_worst

In [ ]:
dfs = {}

for path in paths:
    with open(os.path.join(root, path), "rb") as f:
        dfs[tuple(path.removesuffix(".pkl").split("_")[-2:])] = pkl.load(f)

In [ ]:
dfs_worst = {}

for path in paths_worst:
    with open(os.path.join(root, path), "rb") as f:
        dfs_worst[tuple(path.removesuffix(".pkl").split("_")[-3:-1])] = pkl.load(f)

#### Adj matrix mapping

In [ ]:
adj_mappings = {}

for time in times:
    if time != "all_day_free_flow":
        print_INFO_message(f"Loading adj mapping for {time}")
        path = rf"C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\03_primary\adj_mapping_{time}.pkl"
        with open(path, "rb") as f:
            adj_mappings[time] = dill.load(f)

#### Average graph

In [ ]:
average_graphs = {}

for time in times:
    if time != "all_day_free_flow":
        print_INFO_message(f"Loading adj matrix for {time}")
        path = rf"C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\03_primary\average_graph_{time}.pkl"
        with open(path, "rb") as f:
            average_graphs[time] = pkl.load(f)

In [ ]:
worst_average_graphs = {}

for time in times:
    if time != "all_day_free_flow":
        print_INFO_message(f"Loading adj matrix for {time}")
        path = rf"C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\03_primary\worst_average_graph_{time}.pkl"
        with open(path, "rb") as f:
            worst_average_graphs[time] = pkl.load(f)

### Exact solution analysis

In [ ]:
print_INFO_message_timestamp("Objective value for the Exact solution")
for time, fl_exact in fls_exact.items():
    print_INFO_message(f"{time}: {round(fl_exact.solution_value/60,3)} minutes")

In [ ]:
report_exact = FacilityLocationReport(fls_exact)

In [ ]:
report_exact.graphical_keys_solutions_comparison()

- Let O denote the objective function, t denote time, and x denotes the decision variables. 
- Let x_ff be the value of the decision variables obtained in the free-flow setting. 

For each time condition t, we want to compute the value of the objective function O(x_ff, t) when fixing x to be x_ff.

### Retrieve average graphs

In [ ]:
fls_exact["all_day_free_flow"].locations_coordinates

In [ ]:
sv = fls_exact["all_day_free_flow"].solution_value/60
sv

In [ ]:
a = fls_exact["all_day_free_flow"].adjacency_matrix
print(a.shape)
np.where(a/60 == sv, a, 0).nonzero()

In [ ]:
print(f'source: {adj_mappings["all_day"][1442]}\ndestination: {adj_mappings["all_day"][646]}')

In [ ]:
a = nx.dijkstra_path_length(G=average_graphs["all_day"], 
                            source=adj_mappings["all_day"][1442], 
                            target=adj_mappings["all_day"][646],
                            weight="weight2")/60

b = fls_exact["all_day_free_flow"].adjacency_matrix[1442, 646]/60

print(f"shortest path lenght: {a}\nadj matrix: {b}")

### Compare solution under different scenarios

In [ ]:
def solution_vs_scenario(time_solution, time_scenario, weight="weight2", worst=False):
    
    # Load the exact solution
    print_INFO_message_timestamp(f"Loading exact solution for {time_solution}")
    path = rf"C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\07_model_output\exact_solutions\exact_solution_{time_solution}.pkl"
    fls_exact_solution = FacilityLocation.load(path)
    
    # Load the average graph
    print_INFO_message(f"Loading adj matrix for {time_scenario}")
    if worst:
        path = rf"C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\03_primary\worst_average_graph_{time_scenario}.pkl"
    else:
        path = rf"C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\03_primary\average_graph_{time_scenario}.pkl"
    
    with open(path, "rb") as f:
        average_graph = pkl.load(f)
    
    # extract the coordinates of the exact solution
    ff_solutions_location = fls_exact_solution.locations_coordinates
    
    # compute the distance from the exact solution to all the other nodes in the graph
    print_INFO_message_timestamp(f"Compute the distance from the {time_solution} solution to all the other nodes in the {time_scenario} graph")
    
    temporal_distances = {ff_solutions_location[i].geometry.coords[0]: [] for i in range(len(ff_solutions_location))}

    for i, node in enumerate(average_graph):
        if i%500 == 0:
            print_INFO_message(f"{i} out of {len(average_graph.nodes)}")
            
        keys = list(temporal_distances.keys())
        temporal_distances[keys[0]].append(
            (node, nx.dijkstra_path_length(G=average_graph, 
                                source=keys[0], 
                                target=node,
                                weight=weight))
            ) 
        
        temporal_distances[keys[1]].append(
            (node, nx.dijkstra_path_length(G=average_graph, 
                                source=keys[1], 
                                target=node,
                                weight=weight))
            )
    
    # create a dataframe with the distance from the exact solution to all the other nodes in the graph
    d = {"source": [], "target": [], "travel_time": []}
    
    for key, value in temporal_distances.items():
        for node, distance in value:
            d["source"].append(key)
            d["target"].append(node)
            d["travel_time"].append(round(distance/60, 3))
            
    return pd.DataFrame(d)

In [ ]:
def get_minimum_distances(df):
    return df.groupby("target").min().reset_index()

In [ ]:
keys = list(dfs.keys())
keys

#### Free-flow

In [ ]:
df_all_day_ff_min = get_minimum_distances(dfs[("day", "weight2")])

In [ ]:
df_all_day_ff_min.sort_values(by="travel_time", ascending=False).reset_index().head(5)

In [ ]:
df_all_day_ff_min["travel_time"].mean()

In [ ]:
a = round(fls_exact["all_day_free_flow"].solution_value/60, 3)
b = dfs[("day", "weight2")].groupby("target").min().sort_values(by="travel_time", ascending=False).iloc[0].travel_time

print(f"exact solution: {a}\nff approximation: {b}\nrel_difference: {round(abs(a-b)/a * 100,3)}")

#### All-day

In [ ]:
df_all_day_min = get_minimum_distances(dfs[("day", "weight")])
df_worst_all_day_min = get_minimum_distances(dfs_worst[("day", "weight")])

In [ ]:
print(f'average time: {df_all_day_min["travel_time"].mean()}\naverage worst time: {df_worst_all_day_min["travel_time"].mean()}')

In [ ]:
a = round(fls_exact["all_day"].solution_value/60, 3)

b = df_all_day_min.sort_values(by="travel_time", ascending=False).iloc[0].travel_time
b_worst = df_worst_all_day_min.sort_values(by="travel_time", ascending=False).iloc[0].travel_time

rel_difference_all_day = round(abs(a-b)/a * 100,3)
rel_difference_all_day_worst = round(abs(a-b_worst)/a * 100,3)

print(f"exact solution: {a}\nff approximation: {b}\nrel_difference: {rel_difference_all_day}\nrel_difference_worst: {rel_difference_all_day_worst}")

#### Morning

In [ ]:
df_morning_min = get_minimum_distances(dfs[("morning", "weight")])
df_worst_morning_min = get_minimum_distances(dfs_worst[("morning", "weight")])

In [ ]:
print(f'average time: {df_morning_min["travel_time"].mean()}\naverage worst time: {df_worst_morning_min["travel_time"].mean()}')

In [ ]:
a = round(fls_exact["morning"].solution_value/60, 3)

b = df_morning_min.sort_values(by="travel_time", ascending=False).iloc[0].travel_time
b_worst = df_worst_morning_min.sort_values(by="travel_time", ascending=False).iloc[0].travel_time

rel_difference_morning = round(abs(a-b)/a * 100,3)
rel_difference_morning_worst = round(abs(a-b_worst)/a * 100,3)

print(f"exact solution: {a}\nff approximation: {b}\nrel_difference: {rel_difference_morning}\nrel_difference_worst: {rel_difference_morning_worst}")

#### Midday

In [ ]:
df_midday_min = get_minimum_distances(dfs[("midday", "weight")])
df_worst_midday_min = get_minimum_distances(dfs_worst[("midday", "weight")])

In [ ]:
print(f'average time: {df_midday_min["travel_time"].mean()}\naverage worst time: {df_worst_midday_min["travel_time"].mean()}')

In [ ]:
a = round(fls_exact["midday"].solution_value/60, 3)

b = df_midday_min.sort_values(by="travel_time", ascending=False).iloc[0].travel_time
b_worst = df_worst_midday_min.sort_values(by="travel_time", ascending=False).iloc[0].travel_time

rel_difference_midday = round(abs(a-b)/a * 100,3)
rel_difference_midday_worst = round(abs(a-b_worst)/a * 100,3)

print(f"exact solution: {a}\nff approximation: {b}\nrel_difference: {rel_difference_midday}\nrel_difference_worst: {rel_difference_midday_worst}")

#### Afternoon

In [ ]:
df_afternoon_min = get_minimum_distances(dfs[("afternoon", "weight")])
df_worst_afternoon_min = get_minimum_distances(dfs_worst[("afternoon", "weight")])

In [ ]:
print(f'average time: {df_afternoon_min["travel_time"].mean()}\naverage worst time: {df_worst_afternoon_min["travel_time"].mean()}')

In [ ]:
a = round(fls_exact["afternoon"].solution_value/60, 3)

b = df_afternoon_min.sort_values(by="travel_time", ascending=False).iloc[0].travel_time
b_worst = df_worst_afternoon_min.sort_values(by="travel_time", ascending=False).iloc[0].travel_time

rel_difference_afternoon = round(abs(a-b)/a * 100,3)
rel_difference_afternoon_worst = round(abs(a-b_worst)/a * 100,3)

print(f"exact solution: {a}\nff approximation: {b}\nrel_difference: {rel_difference_afternoon}\nrel_difference_worst: {rel_difference_afternoon_worst}")

### Reporting

In this section, the following steps are performed:
1. **Compare the objective function value** under different scenarios, for a specif set of solution locations
   
2. Given the matrix containing the travel times between all OD pairs, for a specific solution, **compare the distribution** of travel times between OD pairs under different scenarios
   
3. **Dispaly the path** associated to the solutions at step1 

#### Step 1: Compare the objective function value under different scenarios, for a specif set of solution locations

In [ ]:
rel_diffs = [rel_difference_all_day, rel_difference_morning, rel_difference_midday, rel_difference_afternoon]
rel_diffs_worst = [rel_difference_all_day_worst, rel_difference_morning_worst, rel_difference_midday_worst, rel_difference_afternoon_worst]

In [ ]:
fig = make_subplots(rows=1, cols=2,)
fig.update_layout(title="<b>Relative difference between the exact solution and the free flow approximation<b>",
                  title_pad_l=150,
                  height=500,
                  width=1200,
                  yaxis_title="relative difference [%]")

fig.update_yaxes(range=[0, 100])

fig.add_trace(go.Bar(y=rel_diffs, 
                     name="average scenario",
                     x=["all_day", "morning", "midday", "afternoon"],), row=1, col=1)

fig.add_trace(go.Bar(y=rel_diffs_worst,
                     name="average worst scenario",
                     x=["all_day", "morning", "midday", "afternoon"],), row=1, col=2)

#### Step 2: Compare the distribution of travel times between OD pairs under different scenarios

In [ ]:
df_min = df_all_day_ff_min[["target", "travel_time"]]

for df, name in zip([df_all_day_min, df_morning_min, df_midday_min, df_afternoon_min, 
                     df_worst_all_day_min, df_worst_morning_min, df_worst_midday_min, df_worst_afternoon_min], 
                    ["all_day", "morning", "midday", "afternoon", "worst_all_day", "worst_morning", "worst_midday", "worst_afternoon"]):
    
    df_min = df_min.merge(df[["target", "travel_time"]], 
                          on="target", 
                          suffixes=(None, "_"+name),
                          how="outer")

df_min = df_min.rename(columns={"travel_time": "travel_time_free_flow"})

In [ ]:
fig = go.Figure()

show_legend = [True]+[False]*len(df_min.columns[1:])

fig.update_layout(title="<b>Distribution for free flow travel times solution across average scenarios<b>",
                  title_pad_l=150,
                  height=500,
                  width=1200,
                  xaxis_title="time of the day",)

fig.update_yaxes(range=[0, 40])

for i, name in enumerate(["free_flow", "all_day", "morning", "midday", "afternoon",
                          "worst_all_day", "worst_morning", "worst_midday", "worst_afternoon"]):
    fig.add_trace(go.Violin(y=df_min["travel_time_free_flow"],
                            name=name,
                            box_visible=True,
                            meanline_visible=False,
                            hoverinfo="none",
                            side="negative",
                            line_color="lightseagreen",
                            showlegend=show_legend[i]))
    
    fig.add_trace(go.Violin(y=df_min["travel_time_"+name],
                            name=name,
                            box_visible=True,
                            meanline_visible=False,
                            hoverinfo="none",
                            side="positive",
                            line_color="mediumpurple",
                            showlegend=show_legend[-1]))

fig.show()

In [ ]:
# Perform Mann-Whitney U test
for col in df_min.columns[2:]:
    print_INFO_message_timestamp(f"Performing Mann-Whitney U test for {col}")
    statistic, p_value = stats.mannwhitneyu(df_min["travel_time_free_flow"], df_min[col])

    # Print the results
    print_INFO_message(f"Mann-Whitney U statistic: {statistic}")
    print_INFO_message(f"P-value: {p_value}")
    print("\n")

In [ ]:
mean_ci = pd.DataFrame({"mean": None, "lower_bound": None, "upper_bound": None}, 
                       index=df_min.columns[1:])

for col in df_min.columns[1:]:
    # Number of bootstrap iterations
    n_iterations = 1000

    # Confidence level (e.g., 95%)
    confidence_level = 0.95

    # Array to store bootstrap sample statistics
    bootstrap_means = []

    # Perform bootstrap iterations
    for _ in range(n_iterations):
        bootstrap_sample = resample(df_min[col], replace=True, n_samples=len(df_min))
        bootstrap_mean = np.mean(bootstrap_sample)
        bootstrap_means.append(bootstrap_mean)

    # Compute confidence interval
    lower_bound = np.percentile(bootstrap_means, (1 - confidence_level) / 2 * 100)
    upper_bound = np.percentile(bootstrap_means, (1 + confidence_level) / 2 * 100)

    # Add to dataframe
    mean_ci.loc[col] = [df_min[col].mean(), lower_bound, upper_bound]
    
# Print the confidence interval
mean_ci = mean_ci.sort_values(by="mean", ascending=False).round(3)

In [ ]:
mean_ci

In [ ]:
fig = go.Figure()

fig.update_layout(title="<b>Average travel time for free flow solution across average scenarios<b>",
                  title_pad_l=130,
                  height=600,
                  width=1100,
                  xaxis_title="time of the day",
                  yaxis_title="mean travel time [min]")

fig.add_trace(go.Bar(x=mean_ci.index, 
                     y = mean_ci["mean"],
                     width=0.5,
                     name='mean'))

# Add the vertical line
for col in df_min.columns[1:]:
        fig.add_shape(type='line',
                x0=col, y0=mean_ci.loc[col]["lower_bound"],
                x1=col, y1=mean_ci.loc[col]["upper_bound"],
                xref='x', yref='y',
                line=dict(color='red', width=10))

fig.update_yaxes(range=[0, mean_ci["upper_bound"].max()+1])

fig.show()

#### Step 3: Dispaly the path associated to the solutions at step1

In [ ]:
time = "all_day"
print_INFO_message_timestamp(f"Loading gdf for {time}")
path = rf"C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\03_primary\average_{time}.geojson"
with open(path, "rb") as f:
    all_day_gdf = pkl.load(f)

In [ ]:
source, destination = df_all_day_ff_min.sort_values(by="travel_time", ascending=False).iloc[0][["source", "target"]]
solution_path = nx.dijkstra_path(G=average_graphs["all_day"], source=source, target=destination, weight="weight2")

In [ ]:
def get_travel_time(solution_path, graph):
    travel_time = 0
    for i in range(len(solution_path)-1):
        sp = solution_path[i]
        ep = solution_path[i+1]
        travel_time += graph.get_edge_data(sp, ep)["weight"]
    return travel_time

In [ ]:
travel_time = {}

for time in ["free_flow"]+ ['all_day', 'morning', 'midday', 'afternoon']:
    if time == "free_flow":
        travel_time[time] = nx.dijkstra_path_length(G=average_graphs["all_day"], source=source, target=destination, weight="weight2")
    else:
        travel_time[time] = get_travel_time(solution_path, average_graphs[time])
    minutes = int(travel_time[time]/60)
    seconds = int(travel_time[time]%60)
    travel_time[time] = str(minutes) + " min" + " " + str(seconds) + " sec"

In [ ]:
for time in ['all_day', 'morning', 'midday', 'afternoon']:
    travel_time[time+"_worst"] = get_travel_time(solution_path, worst_average_graphs[time])
    minutes = int(travel_time[time+"_worst"]/60)
    seconds = int(travel_time[time+"_worst"]%60)
    travel_time[time+"_worst"] = str(minutes) + " min" + " " + str(seconds) + " sec"

In [ ]:
for time in travel_time.keys():
    print(f"travel time {time}: {travel_time[time]}")

In [ ]:
center_pt = [60.39299, 5.32415]
map = folium.Map(location=center_pt, tiles="OpenStreetMap", zoom_start=11)

tooltip = "<br><br>".join([rf"<b>travel time {time}</b>: " + travel_time[time] for time in ["free_flow"]+
                           ['all_day', 'morning', 'midday', 'afternoon']+
                           ['all_day_worst', 'morning_worst', 'midday_worst', 'afternoon_worst']])

folium.PolyLine(locations=[(node[1], node[0]) for node in solution_path], 
                color="black",
                weight=5,
                tooltip=tooltip).add_to(map)

map

In [ ]:
fls_exact["all_day"].__dict__.keys()